In [1]:
import numpy as np
import random 
## b3hash

In [2]:
def okbithash(val,mod=0,mod2=0):
    mod = f'mod:{mod*mod},{mod2*mod2}'
    s = val + mod
    h = hash(s)
    return(h%2)

In [3]:
byte_count = np.array([0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 4, 5, 5, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8],dtype=np.uint8)

class gbytes(np.ndarray):
    def __new__(cls,b):
        buf = np.frombuffer(b,dtype=np.uint8)
        qa = np.ndarray.__new__(cls,len(b),dtype=np.uint8)
        qa[:] = buf
        return(qa)
    
    def __repr__(self):
        s = 'g\''
        for b in self:
            s += '\\x{:02x}'.format(b)
        s += '\''
        return(s)

    def bit_count(self):
        count = 0
        for b in self:
            count += byte_count[b]
        return(count)

    @staticmethod
    def similarity(gb1, gb2):
        sim_bin = gbytes(np.bitwise_not( np.bitwise_xor( gb1,gb2 ) ))
        sim = sim_bin.bit_count()
        sim = int(sim)
        return(sim)

        

In [4]:
def b3hash(seq,rounds=256):
    out_array = np.ndarray(rounds,dtype=np.bool8)
    seq = ["<START>"] + list(seq) + ["<END>"]
    last = True
    for round in range(rounds):
        sim_sum = 0
        for i in range(1,len(seq)):
            h1 = okbithash(seq[i-1],round,mod2=0)
            h2 = okbithash(seq[i],round,mod2=1)
            x12 = h1 ^ h2
            if x12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)


In [6]:
cats = b3hash('cat')
caats = b3hash('caats')
caaaats = b3hash('caaaats')
catscats = b3hash('catscats')
bcatscats = b3hash('bcatscats')
caattaats = b3hash('caattaats')


In [7]:
elephant = b3hash('elephant')
eleephant = b3hash('eleephant')

In [8]:
alice = b3hash('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april = b3hash('April was finally tired of sitting by her brother by the tree, and of having nothing to do')
ana = b3hash('of sitting by her nothing to do the bank, and of having Ana was beginning sister on to get very tired ')
#ana = b3hash('Ana was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')


long_sentence = b3hash('This is an extremely long sentence which should result in similarity for some reason ok bye')
longsentence = b3hash('Thisisanextremelylongsentencewhichshouldresultinsimilarityforsomereasonokbyesomemorewordshere')

shadowsfollow = b3hash('shadowsfollow')

In [9]:
garbage = b3hash('qwertysjdfniabuioopkjhgfdfvbsdljfnjkasdnxcvbkxjcvdfkjnndmmbyasdkjfbkhbsdfkzxcvibekxcbvkasdljfnkjksbdf')


In [10]:
garbage2 = b3hash(str([random.randint(87,122) for _ in range(5)]))

In [11]:
gbytes.similarity(alice,april) / 256

0.8046875

In [12]:
gbytes.similarity(alice,ana) / 256

0.9453125

In [13]:
gbytes.similarity(april,garbage2) / 256

0.5234375

In [14]:
gbytes.similarity(long_sentence,april) / 256

0.62109375

In [15]:
gbytes.similarity(long_sentence,alice) / 256

0.66796875

In [16]:
gbytes.similarity(garbage,april) / 256

0.50390625

In [17]:
gbytes.similarity(longsentence,long_sentence) / 256

0.75

In [18]:
gbytes.similarity(garbage,longsentence) / 256

0.4765625

In [19]:
gbytes.similarity(caattaats,shadowsfollow) / 256

0.5546875

In [20]:
gbytes.similarity(garbage2,april) / 256

0.5234375

In [21]:
garbage2

g'\x3a\x7a\x55\x31\xb2\x27\xbe\xa3\xf9\x95\x36\x94\x37\xb5\xd2\x8c\x25\xef\x45\x00\x8b\xd0\x6a\xb1\x48\x30\xca\x15\x2d\x18\x2b\xe2'